In [18]:
import numpy as np
from numpy.fft import fft2,fftfreq,fft,fftshift
import os,sys
import yaml
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import cv2
import matplotlib.animation as animation


In [19]:
def wavefront_initialize(pixelsize_h=1e-6,pixelsize_v=1e-6,npixels_h=2048,npixels_v=2048,amplitude_value=0.0):
    #
    #create array at object (aperture) plane
    #
    amplitude = np.zeros((npixels_h,npixels_v))  # amplitude map
    amplitude += amplitude_value
    p_i_h = np.arange(npixels_h) * pixelsize_h
    p_x = (p_i_h - 0.5 * (p_i_h[-1] - p_i_h[0]) )
    p_i_v = np.arange(npixels_v) * pixelsize_v
    p_y = (p_i_v - 0.5 * (p_i_v[-1] - p_i_v[0]) )
    return p_x,p_y,amplitude

In [20]:
def wavefront_aperture(p_x,p_y,amplitude,diameter_h=40e-6,diameter_v=40e-6,diameter_r=40e-6,type=0):
    # aperture_type: 0=circular, 1=Square, 2=Gaussian
    p_xx = p_x[:, np.newaxis]
    p_yy = p_y[np.newaxis, :]

    filter = np.zeros_like(amplitude)
    if type == 0:  # Circular aperture
        radius = (diameter_r/2)
        print("radius=%f um"%(1e6*radius))
        filter_illuminated_indices = np.where(p_xx**2 + p_yy**2 < radius**2)
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    elif type == 1:  # square
        radius_h = (diameter_h/2)
        radius_v = (diameter_v/2)
        print("radius_h=%f um,radius_v=%f um"%(1e6*radius_h,1e6*radius_v))
        filter_illuminated_indices = np.where( (np.abs(p_xx) < radius_h) & (np.abs(p_yy) < radius_v))
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    elif type == 2:  # Gaussian
        sigma = diameter_r/2.35
        print("source sigma=%f um"%(1e6*sigma))
        rho2 = p_xx**2 + p_yy**2
        #TODO: add Gaussian amplitude
        filter = np.sqrt(np.exp(-rho2/2/sigma**2)) # Gaussian in intensity, so srrt for amplitude
        filter = np.exp(-rho2/2/sigma**2) # Gaussian amplitude
    else:
        raise ValueError("Aperture type (shape) not valid")

    return p_x,p_y,amplitude*filter

In [21]:
def propagator2d(x,y,z,method="fraunhofer",wavelength=7.29e-11,propagation_distance=1.0,ap_x=40e-6,ap_y=40e-6,focus_h=0.001,focus_v=0.001,alpha_h=-0.05,alpha_v=-0.05,return_angles=0,magnification = 1):
    #
    # interface to different propagators
    #
    from timeit import default_timer as timer

    t_start = timer()
    if method == "fraunhofer":
        x1,y1,z1 = propagator2d_fraunhoffer(x,y,z,wavelength=wavelength)
        if return_angles:
            pass
        else:
            x1 *= propagation_distance
            #x1 *= 1
            y1 *= propagation_distance
            #y1 *= 1
    elif method == "Focus":
        x1,y1,z1 = focus2d(x,y,z,ap_x=ap_x,ap_y=ap_y,focus_h = focus_x, focus_v = focus_y,alpha_h=alpha_x,alpha_v=alpha_y,wavelength = wavelength)
        if return_angles:
            pass
        else:
            #x1 *= focus_h
            #y1 *= focus_v
            x1 *= 1
            y1 *= 1
    elif method == "fourier_convolution_Fresnel":
        x1,y1,z1 = propagator2d_fourier_convolution(x,y,z,propagation_distance=propagation_distance,wavelength=wavelength)
        if return_angles:
            x1 /= propagation_distance
            y1 /= propagation_distance
    elif method =="fresnel_scaling_propagate":
        x1,y1,z1 = fresnel_scaling_propagator(x,y,z,propagation_distance=1.0,wavelength=1e-10,magnification = 1)
        if return_angles:
            x1 /= propagation_distance
            y1 /= propagation_distance
    else:
        raise Exception("method %s not implemented"%method)
    t_end = timer()
    print("Elapsed time in propagation calculations: %5.3f ms"%((t_end-t_start)*1e3))
    print("Shapes in propagation calculations: before: ",z.shape," after: ",z1.shape)
    print("Limits in propagation calculations H: before: ",x[0],x[-1]," after: ",x1[0],x1[-1]," points: ",x.shape)
    print("Limits in propagation calculations V: before: ",y[0],y[-1]," after: ",y1[0],y1[-1]," points: ",y.shape)
    return x1,y1,z1

In [51]:
def focus2d(p_x,p_y,image,ap_x=40e-6,ap_y=40e-6,focus_h =0.001, focus_v =0.001,alpha_h=-0.05,alpha_v=-0.05,wavelength = 7.29e-11):
    # alpha unit: mrad/rad
    #
    # Consider the lens focusing error on the focus with phase difference
    #

    # frequency for axis 1
    pixelsize = p_x[1] - p_x[0]
    npixels = p_x.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_x = freq_n * freq_nyquist
    freq_x *= wavelength

    # frequency for axis 2
    pixelsize = p_y[1] - p_y[0]
    npixels = p_y.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_y = freq_n * freq_nyquist
    freq_y *= wavelength
    
    freq_xy = np.array(np.meshgrid(freq_y,freq_x))
    # Add lens error into pupil
   
    
    F1 = np.fft.fft2(image)
    F1 = image * np.exp(1.j*np.pi/wavelength*((freq_xy[1])**2/focus_h+(freq_xy[0])**2/focus_v)+1e9j*alpha_h*(freq_xy[1]/focus_v)**3+1e9j*alpha_v*(freq_xy[0]/focus_v)**3)
    
    F1 = fftshift(F1)
    #F1 = np.fft.fft2(F1)
    #F2 = np.fft.fftshift(F1)
    
   # print(freq_xy[1]*focus_h)
    return freq_x,freq_y,F1

In [52]:
def propagator2d_fraunhoffer(x,y,image,wavelength=1e-10):
    """
    Fraunhoffer propagator
    :param x: x array of spatial coordinates in meters
    :param y: y array of spatial coordinates in meters
    :param complax_amplitude array: shape: [n_points_x,n_points_y]
    :param wavelength: photon wavelength in meters
    :return: three arrays with the propagated pattern : angle_x [rad], angle_y, complex_amplitude.
    """
    #
    #compute Fourier transform
    #
    F1 = np.fft.fft2(image)  # Take the fourier transform of the image.
    # Now shift the quadrants around so that low spatial frequencies are in
    # the center of the 2D fourier transformed image.
    F2 = np.fft.fft2( F1 )

    # frequency for axis 1
    pixelsize = p_x[1] - p_x[0]
    npixels = p_x.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_x = freq_n * freq_nyquist
    freq_x *= wavelength

    # frequency for axis 2
    pixelsize = p_y[1] - p_y[0]
    npixels = p_y.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_y = freq_n * freq_nyquist
    freq_y *= wavelength
    
    return freq_x,freq_y,F2

In [53]:
def propagator2d_fourier_convolution(p_x,p_y,image,propagation_distance=1.0,wavelength=1e-10):
    #
    # convolving with the Fresnel kernel via FFT multiplication
    #
    fft = np.fft.fft2(image)

    # frequency for axis 1
    pixelsize = p_x[1] - p_x[0]
    npixels = p_x.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_x = freq_n * freq_nyquist
    # freq = freq * wavelength

    # frequency for axis 2
    pixelsize = p_y[1] - p_y[0]
    npixels = p_y.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_y = freq_n * freq_nyquist
    # freq_y = freq_y * wavelength

    freq_xy = np.array(np.meshgrid(freq_y,freq_x))

    fft *= np.exp((-1.0j) * np.pi * wavelength * propagation_distance *
                  np.fft.fftshift(freq_xy[0]*freq_xy[0] + freq_xy[1]*freq_xy[1]) )

    # fft = np.fft.fftshift(fft)
    # fft *= np.exp((-1.0j) * np.pi * wavelength * propagation_distance *
    #               (freq_xy[0]*freq_xy[0] + freq_xy[1]*freq_xy[1]) )
    # fft = np.fft.ifftshift(fft)

    ifft = np.fft.ifft2(fft)

    return p_x.copy(),p_y.copy(),ifft

In [54]:
def fresnel_scaling_propagator(p_x,p_y,image,propagation_distance=1.0,wavelength=1e-10,magnification = 1):
    #
    # convolving with the Fresnel kernel via FFT multiplication
    #
    fft = np.fft.fft2(image)

    # frequency for axis 1
    pixelsize = p_x[1] - p_x[0]
    npixels = p_x.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_x = freq_n * freq_nyquist
    # freq = freq * wavelength

    # frequency for axis 2
    pixelsize = p_y[1] - p_y[0]
    npixels = p_y.size
    freq_nyquist = 0.5/pixelsize
    freq_n = np.linspace(-1.0,1.0,npixels)
    freq_y = freq_n * freq_nyquist
    # freq_y = freq_y * wavelength

    freq_xy = np.array(np.meshgrid(freq_y,freq_x))
    p_xy = np.array(np.meshgrid(p_y,p_x))
    k = 2*np.pi/wavelength
    
    cof = -1.j * k/2/np.pi/propagation_distance*np.exp(1.j*k*propagation_distance)*np.exp(1.j*k/2/propagation_distance*(p_xy[1]**2+p_xy[0]**2))

    image *= np.exp(1.j*k/2*(p_xy[1]**2+p_xy[0]**2)*magnification/propagation_distance)
    
    fft = np.fft.fft2(image)

    # fft = np.fft.fftshift(fft)
    # fft *= np.exp((-1.0j) * np.pi * wavelength * propagation_distance *
    #               (freq_xy[0]*freq_xy[0] + freq_xy[1]*freq_xy[1]) )
    # fft = np.fft.ifftshift(fft)

    #ifft = np.fft.fft2(fft)

    return p_x.copy(),p_y.copy(),fft

In [55]:
with open('configure.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

ss_size = conf_para['Detector']['ss_size']
fs_size = conf_para['Detector']['fs_size']

pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
wl = conf_para['Source']['wl']
k = 2*np.pi/wl

In [56]:
p_x,p_y,pupil_input = wavefront_initialize(pixelsize_h=1e-7,pixelsize_v=1e-7,npixels_h=2048,npixels_v=2048,amplitude_value=1.0)

In [57]:
p_x,p_y,pupil_lens = wavefront_aperture(p_x,p_y,pupil_input,diameter_h=40e-6,diameter_v=40e-6,type=1)

radius_h=20.000000 um,radius_v=20.000000 um


In [58]:
angle_x,angle_y,pupil_focus = propagator2d(p_x,p_y,pupil_lens,method="Focus",wavelength=wl,ap_x=ap_x,ap_y=ap_y,focus_h=focus_y,focus_v=focus_y,alpha_h=alpha_x,alpha_v=alpha_y,return_angles=1)
#pupil_focus = fftshift(fft2(pupil_lens))

Elapsed time in propagation calculations: 787.126 ms
Shapes in propagation calculations: before:  (2048, 2048)  after:  (2048, 2048)
Limits in propagation calculations H: before:  -0.00010235 0.00010235  after:  -0.0003644999999999912 0.0003644999999999912  points:  (2048,)
Limits in propagation calculations V: before:  -0.00010235 0.00010235  after:  -0.0003644999999999912 0.0003644999999999912  points:  (2048,)


In [59]:
pixelsize_h=1e-7
pixelsize_v=1e-7
npixels_h=2048
npixels_v=2048
w_h = pixelsize_h*npixels_h
w_v = pixelsize_v*npixels_v
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_focus),extent = [x_min,x_max,y_min,y_max],norm=LogNorm())
axes[0].set_title('Amplitude', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_focus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('aperture_input.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
angle_xy = np.array(np.meshgrid(angle_y,angle_x))
z = 1000
F1 = pupil_focus*np.exp(1.j*np.pi/wl/z*((angle_xy[1])**2+(angle_xy[0])**2))
#foucus = np.fft.fft2(F1)
phi = np.pi/wl/z*(angle_xy[1]**2+angle_xy[0]**2)
pupil_def = pupil_lens*np.exp(1.j*phi)
Fpupil = np.fft.fft2(pupil_def)
%matplotlib widget
plt.imshow(np.abs(Fpupil),extent = [x_min,x_max,y_min,y_max],norm=LogNorm())
#plt.plot(phi)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [597]:
p_x /= focus_x
p_y /= focus_y
angle_x,angle_y,pupil_defocus = propagator2d(p_x,p_y,pupil_focus,method="fraunhofer",wavelength=wl,propagation_distance=defocus,return_angles=1)

Elapsed time in propagation calculations: 362.159 ms
Shapes in propagation calculations: before:  (2048, 2048)  after:  (2048, 2048)
Limits in propagation calculations H: before:  -0.08529166666666667 0.08529166666666667  after:  -4.3739999999997537e-07 4.3739999999997537e-07  points:  (2048,)
Limits in propagation calculations V: before:  -0.10235 0.10235  after:  -3.644999999999896e-07 3.644999999999896e-07  points:  (2048,)


In [598]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_defocus),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude with Arrow', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_defocus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase with Arrow', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('defocus.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [599]:
p_x /= defocus
p_y /= defocus
angle_x,angle_y,pupil_det = propagator2d(p_x,p_y,pupil_defocus,method="fraunhofer",wavelength=wl,propagation_distance=det_dist,return_angles=1)

Elapsed time in propagation calculations: 398.032 ms
Shapes in propagation calculations: before:  (2048, 2048)  after:  (2048, 2048)
Limits in propagation calculations H: before:  -21.322916666666668 21.322916666666668  after:  -1.749599999999801e-09 1.749599999999801e-09  points:  (2048,)
Limits in propagation calculations V: before:  -25.5875 25.5875  after:  -1.4579999999998756e-09 1.4579999999998756e-09  points:  (2048,)


In [600]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
p3 = axes[0].imshow(np.abs(pupil_det),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude with Arrow', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(pupil_det)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase with Arrow', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('det.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [605]:
%matplotlib widget
plt.plot(p_x,np.unwrap(np.angle(pupil_defocus[1024])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [612]:
method = "fraunhofer"
if method == "fraunhofer":
        print("Fraunhoffer diffraction valid for distances > > a^2/lambda = %f m " %((ap_x/2)**2/wl/defocus))

Fraunhoffer diffraction valid for distances > > a^2/lambda = 1371.742112 m 


In [613]:
print(ap_x**2/16/wl)

1.3717421124828535


## Wavefront converngence to the focus

In [704]:
p_xnew,p_ynew,wavefront = wavefront_initialize(pixelsize_h=1e-7,pixelsize_v=1e-7,npixels_h=4096*2,npixels_v=4096*2,amplitude_value=1.0)
p_xnew,p_ynew, wavefrontlens= wavefront_aperture(p_xnew,p_ynew,wavefront,diameter_h=40e-6,diameter_v=40e-6,type=1)

radius_h=20.000000 um,radius_v=20.000000 um


In [705]:
f = 1/(1/focus_x + 1/focus_y)
p_xynew = np.array(np.meshgrid(p_ynew,p_xnew))
wavefrontlens = wavefrontlens
#*np.exp(-1.j*k/2/f*(p_xynew[1]**2+p_xynew[0]**2)+1e9j*alpha_x*(p_xynew[1]/f)**3+1e9j*alpha_y*(p_xynew[0]/f)**3)
Fwave = np.fft.fft2(wavefrontlens)

In [699]:
method1 = "fourier_convolution_Fresnel"
p_xdef,p_ydef,wavefront_defocus = propagator2d(p_xnew,p_ynew,wavefrontlens,method= method1 ,wavelength=wl,propagation_distance=defocus,return_angles=0)

Elapsed time in propagation calculations: 19075.351 ms
Shapes in propagation calculations: before:  (8192, 8192)  after:  (8192, 8192)
Limits in propagation calculations H: before:  -0.00040955 0.00040955  after:  -0.00040955 0.00040955  points:  (8192,)
Limits in propagation calculations V: before:  -0.00040955 0.00040955  after:  -0.00040955 0.00040955  points:  (8192,)


In [700]:
method2 ="fresnel_scaling_propagate"
distance = 0.0001
def magnification(R,delta):
    mag = (R+delta)/R
    return mag
mag = magnification(defocus,distance)
p_xdet,p_ydet,wavefront_det = propagator2d(p_xdef,p_ydef,wavefront_defocus,method= method2 ,wavelength=wl,propagation_distance=distance,return_angles=0,magnification=mag)

Elapsed time in propagation calculations: 24190.812 ms
Shapes in propagation calculations: before:  (8192, 8192)  after:  (8192, 8192)
Limits in propagation calculations H: before:  -0.00040955 0.00040955  after:  -0.00040955 0.00040955  points:  (8192,)
Limits in propagation calculations V: before:  -0.00040955 0.00040955  after:  -0.00040955 0.00040955  points:  (8192,)


In [706]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_xnew)
x_max = np.max(p_xnew)
y_min = np.min(p_ynew)
y_max = np.max(p_ynew)
p3 = axes[0].imshow(np.abs(Fwave),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude ', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(Fwave)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase ', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('fresnel_wavefrontdet.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [693]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_xdef)
x_max = np.max(p_xdef)
y_min = np.min(p_ydef)
y_max = np.max(p_ydef)
p3 = axes[0].imshow(np.abs(wavefront_defocus),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude ', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(wavefront_defocus)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase ', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('fresnel_wavefrontdet.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [697]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
x_min = np.min(p_xdet)
x_max = np.max(p_xdet)
y_min = np.min(p_ydet)
y_max = np.max(p_ydet)
p3 = axes[0].imshow(np.abs(wavefront_det),extent = [x_min,x_max,y_min,y_max])
axes[0].set_title('Amplitude ', fontsize=14)
p4 = axes[1].imshow(np.unwrap(np.angle(wavefront_det)),extent = [x_min,x_max,y_min,y_max])
axes[1].set_title('Phase ', fontsize=14)
plt.colorbar(p3,ax = axes[0])
plt.colorbar(p4,ax = axes[1])
plt.savefig('fresnel_wavefrontdet.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …